# Where we left off
In Part 01 we used the `StackAPI` package to query against the Stack Exchange API and subclassed it to limit to Stack Overflow.

In [0]:
"""contents of api.py"""
from stackapi import StackAPI


class StackOverflow(StackAPI):
    """A subclass of `StackAPI` that limits API calls to StackOverflow."""

    def __init__(self, **kwargs):
        super().__init__(name="stackoverflow", **kwargs)

In [1]:
so = StackOverflow()

# fetch some questions
response = so.fetch("questions")
print(response)

{'backoff': 0, 'has_more': True, 'page': 5, 'quota_max': 300, 'quota_remaining': 273, 'total': 0, 'items': [{'tags': ['c#', 'kraken.com'], 'owner': {'reputation': 1, 'user_id': 22158996, 'user_type': 'registered', 'profile_image': 'https://i.stack.imgur.com/rt4Wm.jpg?s=256&g=1', 'display_name': 'style', 'link': 'https://stackoverflow.com/users/22158996/style'}, 'is_answered': False, 'view_count': 25, 'answer_count': 1, 'score': 0, 'last_activity_date': 1688237896, 'creation_date': 1688210354, 'last_edit_date': 1688235475, 'question_id': 76594455, 'content_license': 'CC BY-SA 4.0', 'link': 'https://stackoverflow.com/questions/76594455/c-krakenapi-writing-like-invalid-key-but-key-is-valid', 'title': 'C# KrakenApi writing like &quot;Invalid Key&quot; but key is valid'}, {'tags': ['javascript', 'reactjs', 'api', 'blockchain', 'cryptocurrency'], 'owner': {'reputation': 1, 'user_id': 22160413, 'user_type': 'registered', 'profile_image': 'https://lh3.googleusercontent.com/a/AAcHTtfzXhC65ltWXJ

# Quota
In the responses you may have noticed a couple of keys related to a "quota":

In [2]:
[k for k in response.keys() if k.startswith("quota")]

['quota_max', 'quota_remaining']

These are important:
- `quota_max` tells us the maximum number of API calls we have.
- `quota_remaining` tells us how many we have left.

At the time of writing the `quota_max` is 300. When the `quota_remaining` hits 0 I'll have to wait until it is replenished. To increase our `quota_max` and `quota_remaining` we'll need to register for a free API key. Here's how:
1. Register for a key [here](https://stackapps.com/apps/oauth/register)
2. Copy your `Key` value (should be 24 characters long) and save it somewhere safe.

> NOTE: we won't be needing the Client Secret key for this, but don't go sharing it around as it is exactly that -- a secret.

If you already have a registered StackAPP and want to use that key, you can. To find it go to [stackapps.com/apps/oauth](stackapps.com/apps/oauth), and you'll see all of your registered apps.

# But where do I save my key?
I typically store mine as an environment variable. On Windows you would do the following:
1. Click "Start" (or the Windows key) and search "environment".
2. Click on "Edit environment variables for your account". Do NOT click on "Edit the system environment variables".
3. Click on "New" under the first scroll box.
4. Type a name for the environment variable in the Variable name box (I used "stackapi_key").
5. Paste your KEY value in the Variable value box.
6. Click OK.
7. Click OK again.

Now your API key is stored as an environment variable and can be accessed via the `os.getenv` method. For example:

In [5]:
from os import getenv


# this would print my stackapi_key
# print(getenv("stackapi_key"))

> NOTE: If the above code doesn't print anything you may have to restart your Python session for the changes to take effect. If the value doesn't exist `getenv` will return a default value of `None`.

# I've got the key, what do I do with it?
In the `StackAPI.__init__` method there is an argument called `key`.

In [14]:
help(StackAPI)

Help on class StackAPI in module stackapi.stackapi:

class StackAPI(builtins.object)
 |  StackAPI(name=None, version='2.2', **kwargs)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name=None, version='2.2', **kwargs)
 |      The object used to interact with the Stack Exchange API
 |      
 |      :param name: (string) **(Required)** A valid ``api_site_parameter``
 |          (available from http://api.stackexchange.com/docs/sites) which will
 |          be used to connect to a particular site on the Stack Exchange
 |          Network.
 |      :param version: (float) **(Required)** The version of the API you are connecting to.
 |          The default of ``2.2`` is the current version
 |      :param proxy: (dict) (optional) A dictionary of http and https proxy locations
 |          Example:
 |      
 |          .. code-block:: python
 |      
 |              {'http': 'http://example.com',
 |               'https': 'https://example.com'}
 |      
 |          By default, this is ``

We will set this to our newly stored API key.

In [11]:
# store the key as a variable
key = getenv("stackapi_key")
# supply the `key` to the instance
so = StackAPI(name="stackoverflow", key=key)

# fetch some questions using API key
# `max_quota` and `quota_remaining` should be greater than before
response = so.fetch("questions")
print(response)

{'backoff': 0, 'has_more': True, 'page': 5, 'quota_max': 10000, 'quota_remaining': 9954, 'total': 0, 'items': [{'tags': ['java', 'swing', 'awt'], 'owner': {'reputation': 220, 'user_id': 12913664, 'user_type': 'registered', 'profile_image': 'https://lh3.googleusercontent.com/a-/AAuE7mDlqkjEhz0A2XW-zobCG2WJNw_Zss4fH9Q9X0Zwog=k-s256', 'display_name': 'Dante S.', 'link': 'https://stackoverflow.com/users/12913664/dante-s'}, 'is_answered': False, 'view_count': 1, 'answer_count': 0, 'score': 0, 'last_activity_date': 1688239359, 'creation_date': 1688239359, 'question_id': 76596256, 'content_license': 'CC BY-SA 4.0', 'link': 'https://stackoverflow.com/questions/76596256/place-3-or-more-components-on-top-of-each-other-and-make-them-fit-their-parent', 'title': 'place 3 or more components on top of each other and make them fit their parent'}, {'tags': ['python', 'string', 'function', 'class', 'selenium-webdriver'], 'owner': {'reputation': 11, 'user_id': 15113209, 'user_type': 'registered', 'profil

The `quota_max` and `quota_remaining` are now both _much_ greater than before.

In [15]:
{k: v for k, v in response.items() if k.startswith("quota")}

{'quota_max': 10000, 'quota_remaining': 9954}

Let's bake this in to our source code so we don't have to add the `key` everytime we instantiate our custom `StackOverflow` class.

In [19]:
"""contents of api.py"""
from os import getenv

from stackapi import StackAPI


class StackOverflow(StackAPI):
    """A subclass of `StackAPI` that limits API calls to StackOverflow."""

    def __init__(self, key: str | None = None, **kwargs):
        if key is None:
            key = getenv(key="stackapi_key")
        super().__init__(name="stackoverflow", key=key, **kwargs)

In [18]:
so = StackOverflow()

response = so.fetch("questions")
print(response)

{'backoff': 0, 'has_more': True, 'page': 5, 'quota_max': 10000, 'quota_remaining': 9949, 'total': 0, 'items': [{'tags': ['r', 'vector', 'sequence', 'run-length-encoding'], 'owner': {'reputation': 2459, 'user_id': 17918739, 'user_type': 'registered', 'profile_image': 'https://www.gravatar.com/avatar/2832521254777065786c0e50b2caf987?s=256&d=identicon&r=PG', 'display_name': 'peter861222', 'link': 'https://stackoverflow.com/users/17918739/peter861222'}, 'is_answered': True, 'view_count': 1189, 'accepted_answer_id': 76582508, 'answer_count': 10, 'score': 14, 'last_activity_date': 1688240197, 'creation_date': 1688049990, 'last_edit_date': 1688056913, 'question_id': 76582182, 'content_license': 'CC BY-SA 4.0', 'link': 'https://stackoverflow.com/questions/76582182/find-the-first-of-the-last-1s-sequence', 'title': 'Find the first of the last 1&#39;s sequence'}, {'tags': ['python', 'pyqt', 'resources', 'pyqt6', 'pyrcc'], 'owner': {'reputation': 303, 'user_id': 7799676, 'user_type': 'registered',

Works like a charm! But suppose I wanted to use a different key? Or maybe I don't have the key store in my environment variables -- how do I work around that? I could make `key` an argument with a default value of `None`, and when it is `None` we use the value in our environment variables. This grants us some flexibility.

In [23]:
"""contents of api.py"""
from os import getenv

from stackapi import StackAPI


class StackOverflow(StackAPI):
    """A subclass of `StackAPI` that limits API calls to StackOverflow."""

    def __init__(self, key: str | None = None, **kwargs):
        if key is None:
            key = getenv(key="stackapi_key")
        super().__init__(name="stackoverflow", key=key, **kwargs)

Now I can set `key` to a different value from the one stored in my environment variables.

> NOTE: If you provide a key that isn't registered to an application you will raise an error. For example:

In [27]:
StackOverflow(key="123")

StackAPIError: ('https://api.stackexchange.com/2.2/sites/?pagesize=1000&page=1&filter=%21%2AL1%2AAY-85YllAr2%29&key=123', 400, 'bad_parameter', "`key` doesn't match a known application")